# Using the data

In the folder /data, you can find the relevant input to the notebooks and scripts provided here. The content is described in more detail in the README of the folder. Due to their large size, some files (simulation and topological data) are not provided in the repository, but through the KM3NeT Open Data Centre. With this notebook, you can

- retrieve all available data from the Open Data Centre
- read event information from the VOEvent file
- read and compare the hit distributions of the event

In [1]:
import sys

sys.path.append("../src")

from data_access import download_from_odc, get_event

## Download the full data repository

The full `/data` repository is available for download at the Open Data Centre: https://opendata.km3net.de/dataset.xhtml?persistentId=doi:10.5072/FK2/JW72C9

You can directly download all files that are not yet locally available by calling the download_from_odc function. Provide a filepath 'output_dir' to store the files at a different location than the data folder of the repository.

In [2]:
download_from_odc()

Failed to retrieve dataset metadata: 401


## Reading the event data

The observation results are provided in the [VOEvent](https://voevent.readthedocs.io/en/latest/parse.htmlVOEvent) format and used as reference in these notebooks. To retrieve the values from the file at `/data/event/KM3-230213A_voevent.xml`, use the parsed file:

In [3]:
event = get_event()

for child in event.WhereWhen.ObsDataLocation.ObservationLocation.AstroCoords.Position2D.iterchildren():
    print(child.tag, child.text)

Name1 RA
Name2 Dec
Value2 None
Error2Radius99 3.0
Error2Radius90 2.2
Error2Radius68 1.5
Error2Radius50 1.2


In [4]:
pos = (event.WhereWhen.ObsDataLocation.ObservationLocation.AstroCoords.Position2D.Value2.C1,
       event.WhereWhen.ObsDataLocation.ObservationLocation.AstroCoords.Position2D.Value2.C2)
pos

(94.3, -7.8)

## Read the full hit information

The full hit information is available in the native KM3NeT Root format and a json-parsed version. Both files contain the same information. 

Note that a subset of modules on detection unit (DU) 27 has a preliminary and incomplete orientation calibration, resulting in a rotation of their orientation. When using the hits from the json-file through the provided functions, a correction of an observed timing offset of the hits by 5ns is applied.

### From json

In [5]:
from cherenkov import Cherenkov, emission_points
from readhits import read_zipped_json, merge_multi_hits

fullevent = read_zipped_json("../data/event/KM3-230213A_allhits.json.gz")

fullevent.reconstructed_track

namespace(t=-1232.867378771305,
          pos=Vec(-245.44099312085697, 433.5680178896781, 279.9263124319042),
          dir=Vec(0.9864702733077626, 0.1635640988250655, -0.011098894343689254))

### From ROOT

Or read the ROOT-file using [km3io](https://git.km3net.de/km3py/km3io)

In [6]:
import km3io

In [7]:
event_root = km3io.OfflineReader("../data/event/KM3-230213A_allhits.root")

reco = event_root.tracks[0]

In [8]:
reco[0].t ,reco[0].pos_x, reco[0].pos_y, reco[0].pos_z

(28141366.35162123, -245.44099312085697, 433.5680178896781, 279.9263124319042)